In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
import seaborn as sns
from tqdm import tqdm

In [3]:
pages = np.arange(1, 9951, 50)
headers = {'Accept-Language': 'en-US,en;q=0.8'} 
titles = []
years = []
ratings = []
runtimes = []
imdb_ratings = []
metascores = []
votes = []
genres = []
final_df = pd.DataFrame({'movie': titles,
                      'year': years,
                      'rating': ratings,
                      'genre': genres,
                      'primary genre': [],
                      'runtime_min': runtimes,
                      'imdb': imdb_ratings,
                      'metascore': metascores,
                      'votes': votes}
                      )
genre_list = ['comedy', 'sci-fi', 'horror', 'romance', 'action', 'thriller', 'drama', 'mystery', 'crime',
            'animation', 'adventure', 'fantasy', 'comedy-romance', 'action-comedy', 'superhero']

for gen in tqdm(genre_list):
    for page in tqdm(pages):
        response = get("https://www.imdb.com/search/title?genres=" + str(gen) + "&"
                  + "start="
                  + str(page)
                  + "&explore=title_type,genres&ref_=adv_prv", headers=headers)
        sleep(randint(5,20))
        
        if response.status_code != 200:
            warn("Request: {}; Status Code: {}". format(requests, response.status_code))
        
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        movie_details = page_html.find_all('div', class_ = 'lister-item mode-advanced')
        
        
        for details in movie_details:
            if details.find('div', class_ = 'ratings-metascore') is not None:
                title= details.h3.a.text
                titles.append(title)
                
                if details.h3.find('span', class_= 'lister-item-year text-muted unbold') is not None:
                    year = details.h3.find('span', class_= 'lister-item-year text-muted unbold').text
                    years.append(year)
                    
                else:
                    years.append(None)
                    
                if details.p.find('span', class_ = 'certificate') is not None:
                    
                    rating = details.p.find('span', class_= 'certificate').text
                    ratings.append(rating)
                else:
                    ratings.append("")
                    
                    
                if details.p.find('span', class_ = 'genre') is not None:
                    genre = details.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',')
                    genres.append(genre)

                else:
                    genres.append("")

                if details.p.find('span', class_ = 'runtime') is not None:

                 #runtime
                    time = int(details.p.find('span', class_ = 'runtime').text.replace(" min", "")) 
                    runtimes.append(time)

                else:
                    runtimes.append(None)

                if float(details.strong.text) is not None:

                 #IMDB ratings
                    imdb = float(details.strong.text)
                    imdb_ratings.append(imdb)

                else:
                    imdb_ratings.append(None)

                if details.find('span', class_ = 'metascore').text is not None:

                 #Metascore
                    m_score = int(details.find('span', class_ = 'metascore').text) # make it an integer
                    metascores.append(m_score)

                else:
                    metascores.append(None)

                if details.find('span', attrs = {'name':'nv'})['data-value'] is not None:

                 #Number of votes
                    vote = int(details.find('span', attrs = {'name':'nv'})['data-value'])
                    votes.append(vote)

                else:
                    votes.append(None)
        sci_fi_df = pd.DataFrame({'movie': titles,
                  'year': years,
                  'rating': ratings,
                  'genre': genres,
                   'primary genre': gen,
                  'runtime_min': runtimes,
                  'imdb': imdb_ratings,
                  'metascore': metascores,
                  'votes': votes}
                  )

        sci_fi_df.loc[:, 'year'] = sci_fi_df['year'].str[-5:-1]

        sci_fi_df['n_imdb'] = sci_fi_df['imdb'] * 10
        sci_fi_df = sci_fi_df.loc[sci_fi_df['year'] != 'ovie'] 
        sci_fi_df = sci_fi_df.loc[sci_fi_df['year'] != 'obie']
        sci_fi_df = sci_fi_df.loc[sci_fi_df['year'] != 'ideo']
        
        sci_fi_df.loc[:, 'year'] = pd.to_numeric(sci_fi_df['year'])
            
    final_df = pd.concat([final_df, sci_fi_df], ignore_index=True)
            

100%|██████████| 15/15 [11:42:29<00:00, 2809.95s/it]


#sci_fi_df = sci_fi_df.loc[sci_fi_df['year'] != 'ovie'] 
#sci_fi_df = sci_fi_df.loc[sci_fi_df['year'] != 'obie']
#sci_fi_df.loc[:, 'year'] = pd.to_numeric(sci_fi_df['year'])

In [4]:
final_df.sample(20)

,movie,year,rating,genre,primary genre,runtime_min,imdb,metascore,votes,n_imdb
246452,Roberta,1935.0,Passed,"[Comedy, Musical, Romance]",action-comedy,106.0,7.0,66.0,3503.0,70.0
39482,Despicable Me,2010.0,PG,"[Animation, Adventure, Comedy]",thriller,95.0,7.6,72.0,558718.0,76.0
131118,Banshee Chapter,2013.0,R,"[Horror, Mystery, Sci-Fi]",animation,87.0,5.4,62.0,10406.0,54.0
130152,The Devil's Rejects,2005.0,R,"[Crime, Horror, Western]",animation,107.0,6.7,53.0,102686.0,67.0
173290,To Kill a Mockingbird,1962.0,Approved,"[Crime, Drama]",adventure,129.0,8.3,88.0,321261.0,83.0
163181,Memory,2022.0,R,"[Action, Crime, Thriller]",adventure,114.0,5.7,41.0,29614.0,57.0
23505,Blonde,2022.0,NC-17,"[Biography, Drama, Romance]",romance,167.0,5.5,50.0,65099.0,55.0
262290,Fly Me to the Moon 3D,2007.0,G,"[Animation, Adventure, Family]",action-comedy,84.0,4.5,36.0,4660.0,45.0
134615,Star Wars: Episode I - The Phantom Menace,1999.0,PG,"[Action, Adventure, Fantasy]",animation,136.0,6.5,51.0,821071.0,65.0
261534,Brighton Rock,2010.0,R,"[Crime, Drama, Thriller]",action-comedy,111.0,5.7,57.0,6638.0,57.0


In [5]:
final_df.to_csv("IMDB Movies Data" ,index=False)